# Capstone Project (Mx Data)
## 1. Setup and Reading Files

In [98]:
import pandas as pd
import datetime as dt
import numpy as np
from calendar import monthrange

In [99]:
df = pd.read_csv('mx_raw.csv')
df.head()

,Asset ID,Asset Description,Work ID,Status,Event ID,Event Name,Actual Duration (HRS),date,mx type,mx time,Priority Number
0,LCB3024,Large Canister and Basket Set,MNT-182592,Closed,ME-002747,Semi-Annual Maintenance,0.5,2/28/2018,ME,Semi-Annual,2
1,AB12,Scale,CAL-029177,Closed,CE-002488,Annual Calibration,0.5,9/6/2019,CE,Annual,4
2,AB19,Scale,CAL-024551,Closed,CE-003125,Annual Calibration,0.5,1/31/2018,CE,Annual,4
3,AB2,Scale,CAL-027929,Closed,CE-003394,Annual Calibration,0.5,3/26/2019,CE,Annual,4
4,AB20,Scale,CAL-032226,Closed,CE-002891,Annual Calibration,1.5,11/14/2020,CE,Annual,3


## 2. Find Upcoming Events for Subsequent Month

In [100]:
# convert date column to datetime type
df.date = pd.to_datetime(df.date)

In [101]:
# find unique mx times
df['mx time'].unique()

array(['Semi-Annual', 'Annual', 'Quarterly', 'Monthly', 'Daily',
       'Bi-Weekly', '2 Week', '2yr', 'Weekly'], dtype=object)

In [129]:
# create the new date column by add the date and mx time columns
# convert mx time to hours
# create a list of our conditions
conditions = [
    (df['mx time'] == '2yr'),
    (df['mx time'] == 'Annual'),
    (df['mx time'] == 'Semi-Annual'),
    (df['mx time'] == 'Quarterly'),
    (df['mx time'] == 'Monthly'),
    (df['mx time'] == '2 Week'),
    (df['mx time'] == 'Weekly'),
    (df['mx time'] == 'Bi-Weekly'),
    (df['mx time'] == 'Daily')
    ]

# create a list of the values we want to assign for each condition
values = [17520,
          8760,
          4380,
          2190,
          730,
          336,
          168,
          84,
          24
         ]
# create new date column
df['new_date'] = df.date + pd.to_timedelta(np.select(conditions, values), unit='h')
df['hrs_in_month'] = df.new_date.dt.daysinmonth * 24
df

,Asset ID,Asset Description,Work ID,Status,Event ID,Event Name,Actual Duration (HRS),date,mx type,mx time,Priority Number,new_date,hrs_in_month
0,LCB3024,Large Canister and Basket Set,MNT-182592,Closed,ME-002747,Semi-Annual Maintenance,0.50,2018-02-28,ME,Semi-Annual,2,2018-08-29 12:00:00,744
1,AB12,Scale,CAL-029177,Closed,CE-002488,Annual Calibration,0.50,2019-09-06,CE,Annual,4,2020-09-05 00:00:00,720
2,AB19,Scale,CAL-024551,Closed,CE-003125,Annual Calibration,0.50,2018-01-31,CE,Annual,4,2019-01-31 00:00:00,744
3,AB2,Scale,CAL-027929,Closed,CE-003394,Annual Calibration,0.50,2019-03-26,CE,Annual,4,2020-03-25 00:00:00,744
4,AB20,Scale,CAL-032226,Closed,CE-002891,Annual Calibration,1.50,2020-11-14,CE,Annual,3,2021-11-14 00:00:00,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3631,WMC96,WMT CLAMP BASE,CAL-024288,Closed,CE-002450,Quarterly Maintenance,0.15,2018-01-06,CE,Quarterly,2,2018-04-07 06:00:00,720
3632,WMC97,WMT CLAMP BASE,CAL-032345,Closed,CE-002127,Quarterly Calibration,0.50,2020-12-11,CE,Quarterly,1,2021-03-12 06:00:00,744
3633,WMC98,Wedge Clamp,CAL-027282,Closed,CE-002429,Quarterly Calibration,0.10,2019-01-05,CE,Quarterly,2,2019-04-06 06:00:00,720
3634,WMC98,Wedge Clamp,CAL-024291,Closed,CE-002429,Quarterly Maintenance,0.15,2018-01-06,CE,Quarterly,2,2018-04-07 06:00:00,720


In [122]:
# find today 
today = dt.date.today()

# slice all mx due next month
next_month_mx = df.loc[df.new_date.dt.month == today.month + 1]

# find sum of avg duration
tot_hrs = sum(next_month_mx['Actual Duration (HRS)'])

# display next month mx
next_month_mx.loc[:,['Asset ID', 'Asset Description', 'Work ID', 'Event Name', 'new_date', 'hrs_in_month', 'Priority Number']]

,Asset ID,Asset Description,Work ID,Event Name,new_date,hrs_in_month,Priority Number
16,AB36,Scale,CAL-030719,Annual AB ME3002TE Calibration,2021-04-20 00:00:00,720,4
17,AB37,Scale,CAL-030720,Annual AB ME3002TE Calibration,2021-04-20 00:00:00,720,4
264,ATM2,AlloTrue,MNT-206184,Annual Maintenance,2021-04-23 00:00:00,720,1
274,ATM4,AlloTrue,VAL-000043,ATM 2yr Validation,2021-04-09 00:00:00,720,1
279,AUT10,Autoclave,MNT-206346,Annual Maintenance,2021-04-30 00:00:00,720,1
...,...,...,...,...,...,...,...
3627,WMC94,WMT CLAMP BASE,CAL-024292,Quarterly Maintenance,2018-04-07 06:00:00,720,2
3629,WMC95,WMT CLAMP BASE,CAL-024289,Quarterly Maintenance,2018-04-07 06:00:00,720,2
3631,WMC96,WMT CLAMP BASE,CAL-024288,Quarterly Maintenance,2018-04-07 06:00:00,720,2
3633,WMC98,Wedge Clamp,CAL-027282,Quarterly Calibration,2019-04-06 06:00:00,720,2


## 3. Calculate total MX hours available

In [128]:
next_month_mx.iloc[0, -1]

720

## 4. Can the MX Crew Finish All Events?
#### 3.1 Yes: Write Events to a CSV
#### 3.2 No: Pick Events by Priority and Write Priority Events to a CSV